In [ ]:
# [START drive_search_file]
import io
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

import google.auth

def search_file():
  creds, _ = google.auth.default()

  try:
    # create drive api client
    service = build("drive", "v3", credentials=creds)
    files = []
    page_token = None
    while True:
      # pylint: disable=maybe-no-member
      response = (
          service.files()
          .list(
              q="name contains 'backup itd'",
            #   q="mimeType='image/jpeg'",
              spaces="drive",
              fields="nextPageToken, files(id, name)",
              pageToken=page_token,
          )
          .execute()
      )
      for file in response.get("files", []):
        # Process change
        print(f'Found file: {file.get("name")}, {file.get("id")}')
      files.extend(response.get("files", []))
      page_token = response.get("nextPageToken", None)
      if page_token is None:
        break

  except HttpError as error:
    print(f"An error occurred: {error}")
    files = None

  return files

def delete_file(file_id):
  creds, _ = google.auth.default()

  try:
    # create drive api client
    service = build("drive", "v3", credentials=creds)
    service.files().delete(
        fileId=file_id,
        fields="id",
    ).execute()

  except HttpError as error:
    print(f"An error occurred: {error}")
    files = None

  return files

def download_file(real_file_id):
  """Downloads a file
  Args:
      real_file_id: ID of the file to download
  Returns : IO object with location.

  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  """
  creds, _ = google.auth.default()

  try:
    # create drive api client
    service = build("drive", "v3", credentials=creds)

    file_id = real_file_id

    # pylint: disable=maybe-no-member
    request = service.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      print(f"Download {int(status.progress() * 100)}.")

  except HttpError as error:
    print(f"An error occurred: {error}")
    file = None

  return file.getvalue()

files = search_file()
if len(files) != 1:
    raise Exception("Excepted one file on drive")

io_stream = download_file(real_file_id=files[0].get("id"))
print(io_stream)
# with open("filename.zip????", "w") as f_out:
#     f_out.write(io_stream.getbuffer())



In [ ]:
import pandas as pd
import json
import datetime

In [ ]:
with open(r'C:\Users\Łukasz\Desktop\YT_backup_v2\historia oglądania.json', encoding='UTF-8') as f:
    data = json.load(f)

df = pd.json_normalize(data)
df = df.sort_values('time')
df.groupby('title').count().sort_values('titleUrl', ascending=False)

In [ ]:
df = df.drop(columns=['subtitles', 'products', 'activityControls', 'header'], errors='ignore')
df['fetch_time'] = datetime.datetime.now(datetime.timezone.utc)
df.to_hdf('history_copy.h5', key='df')

In [ ]:
df_load = pd.read_hdf("history_copy.h5")
df_load = pd.concat([df_load, df])
df_load = df_load.drop_duplicates()
df_load